## MEME STOCKS - Are they actually risky or are they worth investing?

#### What is a meme stock?
##### “A meme stock refers to the shares of a company that have gained a cult-like following online and through social media platforms. These online communities can go on to build hype around a stock through narratives and conversations elaborated in discussion threads on websites like Reddit and posts to followers on platforms like Twitter and Facebook.” Investopedia
GME, AMC, TSLA, BBBY, HOOD, BB, PLTR, NOK, SPCE (List of stocks)

#### Comparison securities
SPY, AGG, top 4 (MSFT, AMZN, GOOG, AAPL)


In [1]:
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
from alpaca_trade_api import TimeFrame
import datetime
from dateutil.relativedelta import relativedelta
from pathlib import Path  

%matplotlib inline

In [2]:
load_dotenv()

# Set the variables for the Alpaca API and secret keys
alpaca_api_key = os.getenv('Alpaca_API_Key')
alpaca_secret_key = os.getenv('Alpaca_Secret_Key')

if type(alpaca_api_key) ==type(''):
    if type(alpaca_secret_key) ==type(''):
        print('You have both Alpaca Keys saved in your environment variables')
    else:
        print('You have the Alpaca Key but no Secret Key saved in your environment variables')
else:
    print('You do not have any Alpaca API Keys saved in your environment variables')

You have both Alpaca Keys saved in your environment variables


In [3]:
# Create the Alpaca tradeapi.REST object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version='V2')

In [4]:
# This is where the user inputs are collected and stored as variables

# Set the tickers for both the bond and stock portion of the portfolio
meme_tickers = ['GME','AMC','TSLA','BBBY','HOOD','BB','PLTR','NOK','SPCE']

market_tickers = ['SPY','AGG']

large_cap_tickers = ['AAPL','MSFT','GOOG','AMZN']

# Set the number of years history you'd like the API to pull for the analysis
number_of_years_history = 5

# Set the number of years to look forward in the Monte Carlo forecast
number_of_years_forecast = 5

In [5]:
# Set timeframe to 1D 
timeframe = TimeFrame.Day


number_of_trading_days = 253 * number_of_years_history
number_of_tickers = len(meme_tickers)
limit = number_of_trading_days*number_of_tickers

limit_rows = limit

# Format current date as ISO format
start_date = pd.Timestamp(datetime.date.today() - relativedelta(years=number_of_years_history), tz='America/New_York').isoformat()
end_date = pd.Timestamp(datetime.date.today(), tz='America/New_York').isoformat()

print(f"The approximate number of rows the 'alpaca_prices_df' will produce is: {limit}")

The approximate number of rows the 'alpaca_prices_df' will produce is: 11385


In [6]:
def alpaca_multi_index_df(tickers, tf, start, end, rows):
    stock_prices_multi_index_df = pd.DataFrame()
    count = 0
    for ticker in tickers:
        alpaca_prices = alpaca.get_bars(ticker, tf ,start=start, end=end, limit=rows).df
        column_names = [(ticker, x) for x in alpaca_prices.columns]
        alpaca_prices.columns = pd.MultiIndex.from_tuples(column_names)
        count = count + 1
        if count == 1:
            stock_prices_multi_index_df = alpaca_prices
        else:
            stock_prices_multi_index_df = pd.merge(stock_prices_multi_index_df, alpaca_prices, how = 'outer', left_index=True, right_index=True)
    return stock_prices_multi_index_df


In [7]:
meme_portfolio_multi_index_df = alpaca_multi_index_df(meme_tickers, timeframe, start_date, end_date, limit_rows)
meme_portfolio_multi_index_df

GME                                      \
                              open      high     low   close    volume   
timestamp                                                                
2017-04-05 04:00:00+00:00   22.190   22.2900   21.74   21.76   2168922   
2017-04-06 04:00:00+00:00   21.960   22.1700   21.74   21.84   3296184   
2017-04-07 04:00:00+00:00   21.820   22.0209   21.58   21.66   3754542   
2017-04-10 04:00:00+00:00   21.600   22.1200   21.56   21.88   2345274   
2017-04-11 04:00:00+00:00   21.800   22.1500   21.68   22.11   2718362   
...                            ...       ...     ...     ...       ...   
2022-03-30 04:00:00+00:00  175.025  183.3369  165.00  166.85   9092904   
2022-03-31 04:00:00+00:00  163.100  175.7450  158.51  166.58  11239609   
2022-04-01 04:00:00+00:00  188.690  189.7688  155.26  165.00  13184045   
2022-04-04 04:00:00+00:00  167.000  173.2400  156.51  170.73   4637617   
2022-04-05 04:00:00+00:00  168.000  168.4300  152.00  153.59   3952656   

                                                     AMC                  ...  \
                          trade_count        vwap   open    high     low  ...   
timestamp                                                                 ...   
2017-04-05 04:00:00+00:00       15099   22.014457  31.75  31.800  30.875  ...   
2017-04-06 04:00:00+00:00       16689   21.908465  30.95  31.000  30.350  ...   
2017-04-07 04:00:00+00:00       23473   21.774464  30.70  30.900  30.250  ...   
2017-04-10 04:00:00+00:00       14176   21.901005  30.55  30.700  30.100  ...   
2017-04-11 04:00:00+00:00       16442   22.008212  30.60  30.850  30.350  ...   
...                               ...         ...    ...     ...     ...  ...   
2022-03-30 04:00:00+00:00      175443  174.402192  28.59  29.226  25.350  ...   
2022-03-31 04:00:00+00:00      199225  172.584853  24.77  25.920  23.260  ...   
2022-04-01 04:00:00+00:00      277942  173.455744  25.22  25.280  22.340  ...   
2022-04-04 04:00:00+00:00      102346  166.023376  23.48  23.750  21.940  ...   
2022-04-05 04:00:00+00:00      103818  158.106704  23.18  23.960  21.000  ...   

                                NOK                         SPCE          \
                             volume trade_count      vwap   open    high   
timestamp                                                                  
2017-04-05 04:00:00+00:00  14431265       15669  5.237768    NaN     NaN   
2017-04-06 04:00:00+00:00  10654633       12416  5.248350    NaN     NaN   
2017-04-07 04:00:00+00:00   5663239        7256  5.279580    NaN     NaN   
2017-04-10 04:00:00+00:00   6250937       10213  5.320742    NaN     NaN   
2017-04-11 04:00:00+00:00   8194725       10623  5.258640    NaN     NaN   
...                             ...         ...       ...    ...     ...   
2022-03-30 04:00:00+00:00  25983117       28158  5.559583  10.81  11.140   
2022-03-31 04:00:00+00:00  18098290       21710  5.498655  10.56  10.650   
2022-04-01 04:00:00+00:00  17525572       25566  5.509307   9.98  10.400   
2022-04-04 04:00:00+00:00  14694709       23301  5.572727  10.14  10.685   
2022-04-05 04:00:00+00:00  26475691       25302  5.534666  10.61  10.690   

                                                                             
                              low  close      volume trade_count       vwap  
timestamp                                                                    
2017-04-05 04:00:00+00:00     NaN    NaN         NaN         NaN        NaN  
2017-04-06 04:00:00+00:00     NaN    NaN         NaN         NaN        NaN  
2017-04-07 04:00:00+00:00     NaN    NaN         NaN         NaN        NaN  
2017-04-10 04:00:00+00:00     NaN    NaN         NaN         NaN        NaN  
2017-04-11 04:00:00+00:00     NaN    NaN         NaN         NaN        NaN  
...                           ...    ...         ...         ...        ...  
2022-03-30 04:00:00+00:00  10.460  10.69  13819614.0     82554.0  10.831899  
2022-03-31 04:

In [8]:
meme_portfolio_prices_df = alpaca.get_bars(meme_tickers, timeframe ,start=start_date, end=end_date, limit=limit_rows).df
meme_portfolio_prices_df

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2017-04-05 04:00:00+00:00,31.75,31.8000,30.875,31.00,1227257,7401,31.206448,AMC
2017-04-06 04:00:00+00:00,30.95,31.0000,30.350,30.80,932052,6488,30.741811,AMC
2017-04-07 04:00:00+00:00,30.70,30.9000,30.250,30.35,1099043,6742,30.417316,AMC
2017-04-10 04:00:00+00:00,30.55,30.7000,30.100,30.65,782463,5405,30.393309,AMC
2017-04-11 04:00:00+00:00,30.60,30.8500,30.350,30.70,712543,5323,30.639968,AMC
...,...,...,...,...,...,...,...,...
2022-03-30 04:00:00+00:00,1091.07,1113.9500,1084.000,1093.99,19955007,597501,1098.548645,TSLA
2022-03-31 04:00:00+00:00,1094.01,1103.1399,1076.641,1077.60,16270066,506977,1089.404685,TSLA
2022-04-01 04:00:00+00:00,1080.71,1094.7500,1066.640,1084.59,18057555,548116,1081.406344,TSLA


In [9]:
market_portfolio_multi_index_df = alpaca_multi_index_df(market_tickers, timeframe, start_date, end_date, limit_rows)
market_portfolio_multi_index_df

SPY                                       \
                             open     high      low   close     volume   
timestamp                                                                
2017-04-05 04:00:00+00:00  236.26  237.390  234.540  234.78  108800604   
2017-04-06 04:00:00+00:00  234.96  236.040  234.425  235.44   69135757   
2017-04-07 04:00:00+00:00  235.18  236.000  234.640  235.20   74412311   
2017-04-10 04:00:00+00:00  235.36  236.255  234.730  235.34   67615302   
2017-04-11 04:00:00+00:00  234.90  235.180  233.340  235.06   88045276   
...                           ...      ...      ...     ...        ...   
2022-03-30 04:00:00+00:00  460.29  461.195  456.465  458.79   79668682   
2022-03-31 04:00:00+00:00  457.92  458.760  451.160  452.11  118607132   
2022-04-01 04:00:00+00:00  453.30  453.460  449.140  452.92   88676014   
2022-04-04 04:00:00+00:00  453.20  456.910  452.260  456.80   59663209   
2022-04-05 04:00:00+00:00  455.23  457.830  449.820  451.03   74022189   

                                                      AGG                     \
                          trade_count        vwap    open     high       low   
timestamp                                                                      
2017-04-05 04:00:00+00:00      363080  235.933657  108.48  108.740  108.4530   
2017-04-06 04:00:00+00:00      244996  235.287745  108.64  108.710  108.4800   
2017-04-07 04:00:00+00:00      235941  235.290964  108.80  108.890  108.4200   
2017-04-10 04:00:00+00:00      211985  235.487358  108.53  108.650  108.4901   
2017-04-11 04:00:00+00:00      306525  234.499802  108.79  108.980  108.7500   
...                               ...         ...     ...      ...       ...   
2022-03-30 04:00:00+00:00      659199  459.158702  106.60  107.150  106.5900   
2022-03-31 04:00:00+00:00      751169  455.444107  107.19  107.380  107.0600   
2022-04-01 04:00:00+00:00      624745  451.850848  106.35  106.950  106.2500   
2022-04-04 04:00:00+00:00      503316  455.309774  106.87  106.870  106.5600   
2022-04-05 04:00:00+00:00      644313  453.412462  106.60  106.625  105.7400   

                                                                     
                            close    volume trade_count        vwap  
timestamp                                                            
2017-04-05 04:00:00+00:00  108.69   2304074       11167  108.599103  
2017-04-06 04:00:00+00:00  108.68   1725301        7746  108.588276  
2017-04-07 04:00:00+00:00  108.44   1783599        8534  108.645684  
2017-04-10 04:00:00+00:00  108.58   1903768       12236  108.566615  
2017-04-11 04:00:00+00:00  108.89   2200830       11420  108.874603  
...                           ...       ...         ...         ...  
2022-03-30 04:00:00+00:00  107.15  11544378       24975  107.007713  
2022-03-31 04:00:00+00:00  107.10  13788220       44817  107.229272  
2022-04-01 04:00:00+00:00  106.75  11579091       40126  106.667931  
2022-04-04 04:00:00+00:00  106.84   9956493       32544  106.739298  
2022-04-05 04:00:00+00:00  105.76  11018430       34684  105.931526  

[1260 rows x 14 columns]

In [10]:
market_portfolio_prices_df = alpaca.get_bars(market_tickers, timeframe ,start=start_date, end=end_date, limit=limit_rows).df
market_portfolio_prices_df

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2017-04-05 04:00:00+00:00,108.48,108.740,108.4530,108.69,2304074,11167,108.599103,AGG
2017-04-06 04:00:00+00:00,108.64,108.710,108.4800,108.68,1725301,7746,108.588276,AGG
2017-04-07 04:00:00+00:00,108.80,108.890,108.4200,108.44,1783599,8534,108.645684,AGG
2017-04-10 04:00:00+00:00,108.53,108.650,108.4901,108.58,1903768,12236,108.566615,AGG
2017-04-11 04:00:00+00:00,108.79,108.980,108.7500,108.89,2200830,11420,108.874603,AGG
...,...,...,...,...,...,...,...,...
2022-03-30 04:00:00+00:00,460.29,461.195,456.4650,458.79,79668682,659199,459.158702,SPY
2022-03-31 04:00:00+00:00,457.92,458.760,451.1600,452.11,118607132,751169,455.444107,SPY
2022-04-01 04:00:00+00:00,453.30,453.460,449.1400,452.92,88676014,624745,451.850848,SPY


In [11]:
large_cap_portfolio_multi_index_df = alpaca_multi_index_df(large_cap_tickers, timeframe, start_date, end_date, limit_rows)
large_cap_portfolio_multi_index_df

AAPL                                        \
                             open      high      low   close     volume   
timestamp                                                                 
2017-04-05 04:00:00+00:00  144.64  145.4600  143.810  144.02   27724102   
2017-04-06 04:00:00+00:00  144.26  144.5200  143.450  143.66   21150383   
2017-04-07 04:00:00+00:00  143.73  144.1800  143.270  143.34   16676498   
2017-04-10 04:00:00+00:00  143.50  143.8792  142.900  143.17   18934702   
2017-04-11 04:00:00+00:00  142.90  143.3500  140.060  141.63   30881588   
...                           ...       ...      ...     ...        ...   
2022-03-30 04:00:00+00:00  178.51  179.6100  176.700  177.77   91633724   
2022-03-31 04:00:00+00:00  177.80  178.0300  174.400  174.61  102719625   
2022-04-01 04:00:00+00:00  174.05  174.8800  171.940  174.31   78565894   
2022-04-04 04:00:00+00:00  174.62  178.4900  174.440  178.44   76466048   
2022-04-05 04:00:00+00:00  177.55  178.3000  174.415  175.06   73280511   

                                                     MSFT                     \
                          trade_count        vwap    open     high       low   
timestamp                                                                      
2017-04-05 04:00:00+00:00      169568  144.784955   65.95   66.350   65.4443   
2017-04-06 04:00:00+00:00      124825  143.937203   65.60   66.055   65.4800   
2017-04-07 04:00:00+00:00      110322  143.714195   65.85   65.960   65.4400   
2017-04-10 04:00:00+00:00      112269  143.356350   65.79   65.820   65.3600   
2017-04-11 04:00:00+00:00      194111  141.460993   65.45   65.610   64.8500   
...                               ...         ...     ...      ...       ...   
2022-03-30 04:00:00+00:00      695868  178.119343  313.63  315.950  311.5800   
2022-03-31 04:00:00+00:00      710887  175.876423  314.04  315.140  307.8900   
2022-04-01 04:00:00+00:00      659181  173.413651  309.50  310.130  305.5400   
2022-04-04 04:00:00+00:00      629384  177.485223  310.25  315.110  309.7100   
2022-04-05 04:00:00+00:00      619195  175.978639  313.20  314.865  309.8700   

                           ...     GOOG                              AMZN  \
                           ...   volume trade_count         vwap     open   
timestamp                  ...                                              
2017-04-05 04:00:00+00:00  ...  1555430       23177   835.293199   910.69   
2017-04-06 04:00:00+00:00  ...  1254853       18759   830.487048   913.80   
2017-04-07 04:00:00+00:00  ...  1057262       17571   824.559497   899.65   
2017-04-10 04:00:00+00:00  ...   982180       14777   826.330849   899.63   
2017-04-11 04:00:00+00:00  ...  1079732       19110   821.387738   907.04   
...                        ...      ...         ...          ...      ...   
2022-03-30 04:00:00+00:00  ...  1052319       63872  2854.803811  3370.19   
2022-03-31 04:00:00+00:00  ...  1471847       77471  2812.943930  3326.60   
2022-04-01 04:00:00+00:00  ...  1168379       66961  2804.379684  3282.99   
2022-04-04 04:00:00+00:00  ...   951540       63743  2866.459638  3281.36   
2022-04-05 04:00:00+00:00  ...   961914       64157  2834.469761  3354.83   

                                                                               \
                                high        low    close   volume trade_count   
timestamp                                                                       
2017-04-05 04:00:00+00:00   923.7200   905.6200   909.28  7508516      104374   
2017-04-06 04:00:00+00:00   917.1899   894.4927   898.28  6344076       84652   
2017-04-07 04:00:00+00:00   900.0900   889.3100   894.88  3714372       52488   
2017-04-10 04:00:00+00:00   908.5100   899.0000   907.04  3185728       46059   
2017-04-11 04:00:00+00:00   911.2400   897.5000   902.36  3012770       44779   
...                              ...        ...      ...      ...         ...   
2022-03-30 04:00:00+00:00  3379.0100  3310.0000  

In [12]:
large_cap_portfolio_prices_df = alpaca.get_bars(large_cap_tickers, timeframe ,start=start_date, end=end_date, limit=limit_rows).df
large_cap_portfolio_prices_df

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2017-04-05 04:00:00+00:00,144.64,145.4600,143.81,144.02,27724102,169568,144.784955,AAPL
2017-04-06 04:00:00+00:00,144.26,144.5200,143.45,143.66,21150383,124825,143.937203,AAPL
2017-04-07 04:00:00+00:00,143.73,144.1800,143.27,143.34,16676498,110322,143.714195,AAPL
2017-04-10 04:00:00+00:00,143.50,143.8792,142.90,143.17,18934702,112269,143.356350,AAPL
2017-04-11 04:00:00+00:00,142.90,143.3500,140.06,141.63,30881588,194111,141.460993,AAPL
...,...,...,...,...,...,...,...,...
2022-03-30 04:00:00+00:00,313.63,315.9500,311.58,313.86,28163565,340161,313.713418,MSFT
2022-03-31 04:00:00+00:00,314.04,315.1400,307.89,308.31,33327683,351810,310.414324,MSFT
2022-04-01 04:00:00+00:00,309.50,310.1300,305.54,309.42,27026101,313495,308.049343,MSFT


## Quantitative Analysis

The analysis has several components: performance, risk, risk-return profile, and portfolio diversification.

###  Analyze the Performance

Analyze the data to determine if any of the portfolios outperform the broader stock market (S&P 500, AGG, TOP 4 STOCKS)

In [13]:
#DAILY RETURNS - DANICA
#dailyreturns_df = df.pct_change().dropna()

In [14]:
#CUMULATIVE RETURNS - DANICA
#cumulative_returns = (1+ dailyreturns_df).cumprod() - 1

### Analyze the Risk

Evaluate the risk profile of each portfolio by using the standard deviation and the beta.

In [15]:
meme_portfolio_multi_index_close_df = meme_portfolio_multi_index_df.iloc[:, meme_portfolio_multi_index_df.columns.get_level_values(1)=='close']

In [16]:
meme_portfolio_multi_index_close_df.columns = meme_portfolio_multi_index_close_df.columns.droplevel(1)
meme_portfolio_multi_index_close_df

,GME,AMC,TSLA,BBBY,HOOD,BB,PLTR,NOK,SPCE
timestamp,,,,,,,,,
2017-04-05 04:00:00+00:00,21.76,31.00,295.00,37.800,NaN,NaN,NaN,5.18,NaN
2017-04-06 04:00:00+00:00,21.84,30.80,298.70,39.080,NaN,NaN,NaN,5.23,NaN
2017-04-07 04:00:00+00:00,21.66,30.35,302.54,39.490,NaN,NaN,NaN,5.28,NaN
2017-04-10 04:00:00+00:00,21.88,30.65,312.39,39.380,NaN,NaN,NaN,5.31,NaN
2017-04-11 04:00:00+00:00,22.11,30.70,308.71,39.200,NaN,NaN,NaN,5.27,NaN
...,...,...,...,...,...,...,...,...,...
2022-03-30 04:00:00+00:00,166.85,25.68,1093.99,22.745,14.56,7.59,13.88,5.53,10.69
2022-03-31 04:00:00+00:00,166.58,24.64,1077.60,22.530,13.51,7.46,13.73,5.46,9.88
2022-04-01 04:00:00+00:00,165.00,23.30,1084.59,22.840,13.50,6.75,13.83,5.55,9.99


In [ ]:
filepath = Path('../Project1/App/out.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
market_portfolio_multi_index_df.to_csv(filepath)  

In [ ]:
#BETA/VARIANCE - SUNG

In [ ]:
#STANDARD DEVIATIONS - JUNG

### Analyze the Risk-Return Profile

To determine the overall risk of an asset or portfolio, quantitative analysts and investment managers consider not only its risk metrics but also its risk-return profile. After all, if you have two portfolios that each offer a 10% return but one has less risk, you’d probably invest in the smaller-risk portfolio. For this reason, you need to consider the Sharpe ratios for each portfolio.

In [ ]:
#SHARPE RATIO - JENNIFER

In [ ]:
#MONTE CARLO - RYAN

In [ ]:
# Configure the Monte Carlo simulation for evenly weighted portfolios
# Forecasting 5 years cumulative returns across 500 samples

mc_meme_even = MCSimulation(
    portfolio_data=meme_portfolio_multi_index_df,
#    weights=[.1,.1,.1],
    num_simulation=500,
    num_trading_days=252*number_of_years_forecast)
mc_meme_even.portfolio_data.head()

In [ ]:
mc_meme_even.calc_cumulative_return()

In [ ]:
mc_meme_even.plot_simulation()

In [ ]:
mc_meme_even.plot_distribution()

In [ ]:
mc_meme_even_summary = mc_meme_even.summarize_cumulative_return()
mc_meme_even_summary

In [ ]:
mc_meme_even_lower_95 = mc_meme_even_summary[8]
mc_meme_even_upper_95 = mc_meme_even_summary[9]
mc_meme_even_mean = mc_meme_even_summary[1]
display(round(mc_meme_even_lower_95,2),
        round(mc_meme_even_mean,2),
        round(mc_meme_even_upper_95,2))

In [ ]:
# Configure the Monte Carlo simulation for evenly weighted portfolios
# Forecasting 5 years cumulative returns across 500 samples

mc_market_even = MCSimulation(
    portfolio_data=market_portfolio_multi_index_df,
#    weights=[.5,.5],
    num_simulation=500,
    num_trading_days=252*number_of_years_forecast)
mc_meme_even.portfolio_data.head()

In [ ]:
mc_market_even.calc_cumulative_return()

In [ ]:
mc_market_even.plot_simulation()

In [ ]:
mc_market_even.plot_distribution()

In [ ]:
mc_market_even_summary = mc_market_even.summarize_cumulative_return()
mc_market_even_summary

In [ ]:
mc_market_even_lower_95 = mc_market_even_summary[8]
mc_market_even_upper_95 = mc_market_even_summary[9]
mc_market_even_mean = mc_market_even_summary[1]
display(round(mc_market_even_lower_95,2),
        round(mc_market_even_mean,2),
        round(mc_market_even_upper_95,2))

In [ ]:
# Configure the Monte Carlo simulation for evenly weighted portfolios
# Forecasting 5 years cumulative returns across 500 samples

mc_large_cap_even = MCSimulation(
    portfolio_data=large_cap_portfolio_multi_index_df,
#    weights=[.25,.25,.25,.25],
    num_simulation=500,
    num_trading_days=252*number_of_years_forecast)
mc_meme_even.portfolio_data.head()

In [ ]:
mc_large_cap_even.calc_cumulative_return()

In [ ]:
mc_large_cap_even.plot_simulation()

In [ ]:
mc_large_cap_even.plot_distribution()

In [ ]:
mc_large_cap_even_summary = mc_large_cap_even.summarize_cumulative_return()
mc_large_cap_even_summary

In [ ]:
mc_large_cap_even_lower_95 = mc_large_cap_even_summary[8]
mc_large_cap_even_upper_95 = mc_large_cap_even_summary[9]
mc_large_cap_even_mean = mc_large_cap_even_summary[1]
display(round(mc_large_cap_even_lower_95,2),
        round(mc_large_cap_even_mean,2),
        round(mc_large_cap_even_upper_95,2))

In [ ]:
print(f"Meme stock portfolio average return is: {round(100*mc_meme_even_mean,1)}%")
print(f"Large Cap portfolio average return is: {round(100*mc_large_cap_even_mean,1)}%")
print(f"S&P 500 and Bond market average return is: {round(100*mc_market_even_mean,1)}%")

In [ ]:
print("There is a 95% chance the return will be between:")
print(f"Meme stock portfolio lower return is: {round(100*mc_meme_even_lower_95,1)}% - {round(100*mc_meme_even_upper_95,1)}%")
print(f"Large Cap portfolio lower return is: {round(100*mc_large_cap_even_lower_95,1)}% - {round(100*mc_large_cap_even_upper_95,1)}%")
print(f"S&P 500 and Bond market lower return is: {round(100*mc_market_even_lower_95,1)}% - {round(100*mc_market_even_upper_95,1)}%")

### Diversify the Portfolio

Based on our analysis so far, we have the answer to the question, "Are the meme stocks worth investing in?"